In [1]:
import torch
import cppcuda_tutorial
import time

In [2]:
from interpolation import trilinear_interpolation_py, Trilinear_interpolation_cuda
# import interpolation

In [3]:
N = 65536
F = 256

In [4]:
rand = torch.rand([N, 8, F]).cuda()
feats = rand.clone().requires_grad_()
feats2 = rand.clone().requires_grad_()
points = torch.rand(N, 3).cuda() * 2 - 1

In [5]:
t = time.time()
out_cuda = Trilinear_interpolation_cuda.apply(feats2, points)
torch.cuda.synchronize()
print('cuda forward time', time.time() - t, 's')

cuda forward time 0.002151012420654297 s


In [6]:
t = time.time()
out_py = trilinear_interpolation_py(feats, points)
torch.cuda.synchronize()
print("pytorch forward time", time.time() -t, 's')

pytorch forward time 0.007079601287841797 s


In [7]:
print('forward', torch.allclose(out_cuda, out_py))

forward True


In [8]:
'''
Author: BigCiLeng && bigcileng@outlook.com
Date: 2023-10-29 17:49:13
LastEditors: BigCiLeng && bigcileng@outlook.com
LastEditTime: 2023-10-29 19:01:11
FilePath: \cppcuda\test.ipynb
Description: 

Copyright (c) 2023 by bigcileng@outlook.com, All Rights Reserved. 
'''
t = time.time()
loss2 = out_cuda.sum()
loss2.backward()
torch.cuda.synchronize()
print('cuda backward time', time.time() - t, 's')

cuda backward time 0.006810426712036133 s


In [9]:
t = time.time()
loss = out_py.sum()
loss.backward()
torch.cuda.synchronize()
print('pytorch backward time', time.time() - t, 's')

pytorch backward time 0.03897571563720703 s


In [10]:
print('backward', torch.allclose(feats.grad, feats2.grad))

backward True
